In [ ]:
import numpy as np
import pandas as pd

class DecisionTreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature      # Feature index
        self.threshold = threshold  # Threshold (for continuous) or value for categorical
        self.left = left            # Left subtree
        self.right = right          # Right subtree
        self.value = value          # Prediction at leaf node


class DecisionTree:
    def __init__(self, criterion='entropy', max_depth=None, min_samples_split=2, gain_ratio=False):
        self.criterion = criterion          # 'entropy' or 'gini'
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.gain_ratio = gain_ratio
        self.root = None

    # 🎯 1. Fit method
    def fit(self, X, y):
        self.root = self._grow_tree(X, y, depth=0)

    # 🎯 2. Prediction
    def predict(self, X):
        return [self._traverse_tree(x, self.root) for x in X]

    # 🎯 3. Traverse the tree
    def _traverse_tree(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)

    # 🎯 4. Build the tree recursively
    def _grow_tree(self, X, y, depth):
        num_samples, num_features = X.shape
        num_classes = len(np.unique(y))

        # 📍 Stopping criteria
        if num_classes == 1 or num_samples < self.min_samples_split or (self.max_depth and depth >= self.max_depth):
            leaf_value = self._majority_class(y)
            return DecisionTreeNode(value=leaf_value)

        # 📍 Find the best split
        best_feature, best_threshold = self._best_split(X, y)

        # If can't split further
        if best_feature is None:
            leaf_value = self._majority_class(y)
            return DecisionTreeNode(value=leaf_value)

        # 📍 Split into left and right subsets
        left_idxs = X[:, best_feature] <= best_threshold
        right_idxs = X[:, best_feature] > best_threshold

        left_subtree = self._grow_tree(X[left_idxs], y[left_idxs], depth+1)
        right_subtree = self._grow_tree(X[right_idxs], y[right_idxs], depth+1)

        return DecisionTreeNode(
            feature=best_feature,
            threshold=best_threshold,
            left=left_subtree,
            right=right_subtree
        )

    # 🎯 5. Choose the best feature to split on
    def _best_split(self, X, y):
        num_features = X.shape[1]
        best_gain = -np.inf
        split_idx, split_thr = None, None

        for feature in range(num_features):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                gain = self._information_gain(y, X[:, feature], threshold)
                if self.gain_ratio:
                    gain /= self._split_info(X[:, feature], threshold)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feature
                    split_thr = threshold

        return split_idx, split_thr

    # 🎯 6. Compute Information Gain
    def _information_gain(self, y, X_col, threshold):
        parent_criterion = self._impurity(y)

        left_idxs = X_col <= threshold
        right_idxs = X_col > threshold
        
        if sum(left_idxs)==0 or sum(right_idxs)==0:
            return 0

        n = len(y)
        n_left, n_right = sum(left_idxs), sum(right_idxs)

        left_criterion = self._impurity(y[left_idxs])
        right_criterion = self._impurity(y[right_idxs])
        child_criterion = (n_left/n)*left_criterion + (n_right/n)*right_criterion

        return parent_criterion - child_criterion

    # 🎯 7. Compute Split Info (for gain ratio)
    def _split_info(self, X_col, threshold):
        left_idxs = X_col <= threshold
        right_idxs = X_col > threshold
        n = len(X_col)
        p_left = sum(left_idxs)/n
        p_right = sum(right_idxs)/n
        split_info = 0
        for p in [p_left, p_right]:
            if p > 0:
                split_info -= p * np.log2(p)
        return split_info if split_info != 0 else 1e-9

    # 🎯 8. Impurity (Entropy or Gini)
    def _impurity(self, y):
        probs = np.bincount(y) / len(y)
        if self.criterion == 'entropy':
            return -np.sum([p*np.log2(p) for p in probs if p>0])
        elif self.criterion == 'gini':
            return 1 - sum(p**2 for p in probs)

    # 🎯 9. Majority class
    def _majority_class(self, y):
        return np.bincount(y).argmax()



In [4]:
# Dummy dataset
X = np.array([[2.7, 2.5], [1.0, 1.1], [3.0, 3.0], [2.0, 1.5], [3.5, 2.5]])
y = np.array([0, 1, 0, 1, 0])

# Fit a tree
tree = DecisionTree(criterion='gini', max_depth=3, min_samples_split=2, gain_ratio=True)
tree.fit(X, y)

# Predictions
print(tree.predict(X))


[np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0)]
